In [36]:
import numpy as np
from keras.layers import Embedding, Flatten, Dense
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.utils import pad_sequences
docs = ['additional income',
'best price',
'big bucks',
'cash bonus',
'earn extra cash',
'spring savings certificate',
'valero gas marketing',
'all domestic employees',
'nominations for oct',
'confirmation from spinner']

### one_hot() 함수는 각 단어를 고유한 정수로 인코딩하는 과정에서, 단어의 해시 값을 사용하여 정수로 변환한다. 해시 함수는 주어진 단어를 고정된 크기의 정수 범위로 매핑하는데, 이 과정에서 서로 다른 단어가 동일한 해시 값으로 매핑해서 출돌이 일어나 동일한 해시값으로 매필된 단어들이 같은 정수로 인코딩될수 있는 문제가 발생할 수 있다.

In [37]:
labels = np.array([1,1,1,1,1,0,0,0,0,0])
vocab_size = 50
encoded_docs = [one_hot(d, vocab_size) for d in docs]
print(encoded_docs)

[[11, 10], [26, 44], [14, 44], [15, 1], [31, 8, 15], [17, 39, 47], [26, 27, 22], [20, 15, 36], [24, 7, 15], [9, 39, 40]]


### 해시 충돌로 인해 발생하는 이런 문제를 해결하기 위해서는 해싱 대신에 다른 단어를 인코딩 하는 방법을 사용해야 한다.
### 대표적으로는 단어를 정수로 매핑하는 단어 집합을 만들고 각 단어에 고유한 정수를 할당하는 방법이 있다.

In [38]:
from tensorflow.keras.preprocessing.text import Tokenizer

t= Tokenizer()
t.fit_on_texts(docs)
print("단어 집합 : ",t.word_index,"\n")

encoded_docs = t.texts_to_sequences(docs)
print("texts_to_sequences : ",encoded_docs,"\n")

vocab_size = len(t.word_index) + 1 # 1을 더해주는 이유는 1부터 시작하기 위해서

max_length =3

padded_docs= pad_sequences(encoded_docs,maxlen=max_length,padding='post')
print('shape = ',padded_docs.shape,"\n")
print(padded_docs,"\n")



단어 집합 :  {'cash': 1, 'additional': 2, 'income': 3, 'best': 4, 'price': 5, 'big': 6, 'bucks': 7, 'bonus': 8, 'earn': 9, 'extra': 10, 'spring': 11, 'savings': 12, 'certificate': 13, 'valero': 14, 'gas': 15, 'marketing': 16, 'all': 17, 'domestic': 18, 'employees': 19, 'nominations': 20, 'for': 21, 'oct': 22, 'confirmation': 23, 'from': 24, 'spinner': 25} 

texts_to_sequences :  [[2, 3], [4, 5], [6, 7], [1, 8], [9, 10, 1], [11, 12, 13], [14, 15, 16], [17, 18, 19], [20, 21, 22], [23, 24, 25]] 

shape =  (10, 3) 

[[ 2  3  0]
 [ 4  5  0]
 [ 6  7  0]
 [ 1  8  0]
 [ 9 10  1]
 [11 12 13]
 [14 15 16]
 [17 18 19]
 [20 21 22]
 [23 24 25]] 



* fit_on_texts()는 Tokenizer 클래스의 메서드로, 주어진 텍스트 데이터를 기반으로 단어 집합을 생성하는 역할을 한다. 이 메서드는 입력으로 리스트 형태의 텍스트 데이터를 받는다. 이 데이터를 사용하여 Tokenizer 객체 내부의 상태를 업데이트하고, 단어 집합을 구축한다. 각 텍스트를 단어로 분리되고, 이 단어들을 기반으로 단어 집합이 형성된다.

* word_index는 Tokenizer 객체의 속성으로, 단어와 해당 단어의 정수 인덱스를 매핑한 딕셔너리이다.

In [39]:
model = Sequential()
model.add(Embedding (vocab_size,8,input_length = max_length))
model.add(Flatten())
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(padded_docs,labels,epochs=50,verbose=1)

loss,accuracy=model.evaluate(padded_docs, labels,verbose=1)

print('정확도 = ',accuracy)

Epoch 1/50
1/1 [==============================] - 1s 964ms/step - loss: 0.6963 - accuracy: 0.5000
Epoch 2/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6934 - accuracy: 0.6000
Epoch 3/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6906 - accuracy: 0.6000
Epoch 4/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6878 - accuracy: 0.7000
Epoch 5/50
1/1 [==============================] - 0s 9ms/step - loss: 0.6850 - accuracy: 0.7000
Epoch 6/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6822 - accuracy: 0.7000
Epoch 7/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6795 - accuracy: 0.8000
Epoch 8/50
1/1 [==============================] - 0s 5ms/step - loss: 0.6767 - accuracy: 0.8000
Epoch 9/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6740 - accuracy: 0.9000
Epoch 10/50
1/1 [==============================] - 0s 7ms/step - loss: 0.6712 - accuracy: 0.9000
Epoch 11/50
1/1 [=================

* model = Sequential() 모델 객체를 생성한다. Sequential 모델은 각 레이어를 순차적으로 쌓아 구성하는 방식이다.
* model.add(Embedding (vocab_size,8,input_length = max_length)) : Embedding 레이어를 모델에 추가한다. Embedding은 단어를 밀집 벡터로 변환하는 역할을 한다.
* model.add(Flatten()) : Flatten 레이어를 모델에 추가한다. Flatten은 다차원 입력을 1차원으로 변환한다. Embedding레이어의 출력을 Flatten 레이어로 전달하기 위해 사용된다.
* model.add(Dense(1,activation='sigmoid')) : Dense 레이어를 모델에 추가한다. Dense 레이어는 fully connected 레이어를 의미하며, 1개의 뉴런과 sigmoid 활성화 함수를 가지고 있고, 이 레이어는 이진 분류를 위해 사용된다.
* model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy']) : 모델을 컴파일한다.'adam' 옵티마이저를 사용하고, 손실 함수로는 이진 분류에서 자주 사용되는 'binary_crossentropy'를 선택한다. 정확도를 측정하기 위해 'accuracy' 메트릭을 사용한다
* model.fit(padded_docs,labels,epochs=50,verbose=0) : 모델을 학습시킨다. padded_docs는 패딩이 적용된 입력 데이터, labels는 해당하는 레이블 데이터이다. epochs = 50 는 전체 데이터셋을 50번 반복하여 학습하는 것을 의미한다. verbose = 0 은 학습과정의 로그를 출력하지 않음을 의미한다.
* loss,accuracy=model.evaluate(padded_docs, labels,verbose=1) : 학습된 모델을 평가한다. padded_docs와 labels를 이용하여 모델의 손실과 정확도를 평가한다.

In [43]:
test_doc= ['big income','income how']

t= Tokenizer()
t.fit_on_texts(test_doc)
print("단어 집합 : ",t.word_index,"\n")

encoded_docs = t.texts_to_sequences(test_doc)
print("texts_to_sequences : ",encoded_docs,"\n")

vocab_size = len(t.word_index) + 1 # 1을 더해주는 이유는 1부터 시작하기 위해서

max_length =3

padded_docs= pad_sequences(encoded_docs,maxlen=max_length,padding='post')
print('shape = ',padded_docs.shape,"\n")
print(padded_docs,"\n")


단어 집합 :  {'income': 1, 'big': 2, 'how': 3} 

texts_to_sequences :  [[2, 1], [1, 3]] 

shape =  (2, 3) 

[[2 1 0]
 [1 3 0]] 



In [44]:
model.predict(padded_docs)

1/1 [==============================] - 0s 17ms/step


array([[0.54078496],
       [0.5589574 ]], dtype=float32)

In [45]:
padded_docs

array([[2, 1, 0],
       [1, 3, 0]])